In [1]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline 
import math
import copy
import os
from mayavi import mlab

In [83]:
from semantic_kitti_api_laserscan import *
import yaml

CFG = yaml.safe_load(open("semantic-kitti-api/config/semantic-kitti.yaml", 'r'))
path = "semantic_kitti_data"
color_dict = CFG["color_map"]
nclasses = len(color_dict)
scan = SemLaserScan(nclasses, color_dict, project=True)
poses = np.loadtxt("semantic_kitti_data/sequences/01/poses.txt")
poses = poses.reshape(-1,3,4)

def transform_mat(_pts, pose):
    # multiply matrix of 3d points by transformation matrix, which will result in original 3d points
    # being synchronized across multiple sequences, meaning a rigid object will stay in place even though
    # the LIDAR is moving
    calibration_tr = np.array([4.276802385584e-04, -9.999672484946e-01, -8.084491683471e-03, -1.198459927713e-02, 
                               -7.210626507497e-03, 8.081198471645e-03, -9.999413164504e-01, -5.403984729748e-02, 
                               9.999738645903e-01, 4.859485810390e-04, -7.206933692422e-03, -2.921968648686e-01,
                               0, 0, 0, 1])
    tr = calibration_tr.reshape(4,4)
    tr_inv = np.linalg.inv(tr)
    pose = np.matmul(tr_inv, np.matmul(pose, tr))
    n, _ = _pts.shape
    x = np.hstack((_pts, np.ones((n, 1))))
    x = np.matmul(pose, x.transpose()).transpose()
    return x[:, 0:3]


def get_frame_ground_only(number):
    file_name = "0" * int(6 - (len(str(number)))) + str(number)
    scan = np.fromfile("semantic_kitti_data/sequences/01/velodyne/" + file_name + ".bin", dtype=np.float32)
    labels = np.fromfile("semantic_kitti_data/sequences/01/labels/" + file_name + ".label", dtype=np.uint32)
    scan = scan.reshape((-1, 4))
    intensities = scan[:,3]
    pts = scan[:,0:3]
    
    pose = poses[number]
    pose = np.vstack((pose, [0,0,0,1]))
    pts = transform_mat(pts, pose).reshape(-1,3)
    
    mask = np.array(labels == 40) | np.array(labels == 72) | np.array(labels == 44)\
        | np.array(labels == 48) | np.array(labels == 49)
    pts_without_ground = pts[mask]
    intensities = intensities[mask]
    return pts_without_ground, intensities

def get_frame(number):
    file_name = "0" * int(6 - (len(str(number)))) + str(number)
    scan = np.fromfile("semantic_kitti_data/sequences/01/velodyne/" + file_name + ".bin", dtype=np.float32)
    labels = np.fromfile("semantic_kitti_data/sequences/01/labels/" + file_name + ".label", dtype=np.uint32)
    scan = scan.reshape((-1, 4))
    intensities = scan[:,3]
    pts = scan[:,0:3]
    
    pose = poses[number]
    pose = np.vstack((pose, [0,0,0,1]))
    pts = transform_mat(pts, pose).reshape(-1,3)
    
    mask = np.array(labels != 40) & np.array(labels != 72) & np.array(labels != 44)\
        & np.array(labels != 48) & np.array(labels != 49)
    pts_without_ground = pts[mask]
    intensities = intensities[mask]
    return pts_without_ground, intensities

def get_frame_with_ground(number):
    file_name = "0" * int(6 - (len(str(number)))) + str(number)
    scan = np.fromfile("semantic_kitti_data/sequences/01/velodyne/" + file_name + ".bin", dtype=np.float32)
    labels = np.fromfile("semantic_kitti_data/sequences/01/labels/" + file_name + ".label", dtype=np.uint32)
    scan = scan.reshape((-1, 4))
    intensities = scan[:,3]
    pts = scan[:,0:3]
    
    pose = poses[number]
    pose = np.vstack((pose, [0,0,0,1]))
    pts = transform_mat(pts, pose).reshape(-1,3)
    
    mask = np.array(labels != 40) & np.array(labels != 72) & np.array(labels != 44)\
        & np.array(labels != 48) & np.array(labels != 49)
    pts_without_ground = pts
    intensities = intensities
    return pts_without_ground, intensities

In [85]:
ground_pts, ground_intens = get_frame_ground_only(10)
#mlab.points3d(pts1[:,0], pts1[:,1], pts1[:,2], mode='cube', scale_mode = 'none', scale_factor=0.05, color=(1,0,0))
mlab.points3d(pts1[:,0], pts1[:,1], pts1[:,2], intens, mode='point')

mlab.show()

In [86]:
pts1, intens = get_frame_with_ground(10)
#mlab.points3d(pts1[:,0], pts1[:,1], pts1[:,2], mode='cube', scale_mode = 'none', scale_factor=0.05, color=(1,0,0))
mlab.points3d(pts1[:,0], pts1[:,1], pts1[:,2], intens, mode='point')

mlab.show()

In [87]:
pts_without_ground, intens_without_ground = get_frame(10)


In [75]:
# https://medium.com/@ajithraj_gangadharan/3d-ransac-algorithm-for-lidar-pcd-segmentation-315d2a51351
import random
def ransac(points, max_iterations, distance_ratio_threshold, min_inliers_to_pass):
    inliers_result = []
    outliers_result = []

    for _ in range(max_iterations):
        # Add 3 random indexes
        random.seed()
        inliers = []
        outliers = []
        n, _ = points.shape
        while len(inliers) < 3:
            random_index = random.randint(0, n)
            inliers.append(random_index)

        x1, y1, z1 = points[inliers[0]]
        x2, y2, z2 = points[inliers[1]]
        x3, y3, z3 = points[inliers[2]]
        # Plane Equation --> ax + by + cz + d = 0
        # Value of Constants for inlier plane
        a = (y2 - y1) * (z3 - z1) - (z2 - z1) * (y3 - y1)
        b = (z2 - z1) * (x3 - x1) - (x2 - x1) * (z3 - z1)
        c = (x2 - x1) * (y3 - y1) - (y2 - y1) * (x3 - x1)
        d = -(a * x1 + b * y1 + c * z1)
        plane_lenght = max(0.1, math.sqrt(a * a + b * b + c * c))

        for idx, point in enumerate(points):
            if idx in inliers:
                # point already used
                continue
            x, y, z = point

            # Calculate the distance of the point to the inlier plane
            distance = math.fabs(a * x + b * y + c * z + d) / plane_lenght
            # Add the point as inlier, if within the threshold distancec ratio
            if distance <= distance_ratio_threshold:
                inliers.append(idx)
            else:
                outliers.append(idx)

        # Update the set for retaining the maximum number of inlier points
        if len(inliers) > len(inliers_result):
            inliers_result = inliers
            outliers_result = outliers

        if len(inliers) >= min_inliers_to_pass:
            break
    # Segregate inliers and outliers from the point cloud
    #inlier_points = points[inliers_result]
    #outlier_points = points[outliers_result]

    return inliers_result, outliers_result

In [76]:
# https://stackoverflow.com/questions/38754668/plane-fitting-in-a-3d-point-cloud
def PCA(data, correlation=False, sort=True):
    """ Applies Principal Component Analysis to the data

Parameters
----------        
data: array
    The array containing the data. The array must have NxM dimensions, where each
    of the N rows represents a different individual record and each of the M columns
    represents a different variable recorded for that individual record.
        array([
        [V11, ... , V1m],
        ...,
        [Vn1, ... , Vnm]])

correlation(Optional) : bool
        Set the type of matrix to be computed (see Notes):
            If True compute the correlation matrix.
            If False(Default) compute the covariance matrix. 
            
sort(Optional) : bool
        Set the order that the eigenvalues/vectors will have
            If True(Default) they will be sorted (from higher value to less).
            If False they won't.   
Returns
-------
eigenvalues: (1,M) array
    The eigenvalues of the corresponding matrix.
    
eigenvector: (M,M) array
    The eigenvectors of the corresponding matrix.

Notes
-----
The correlation matrix is a better choice when there are different magnitudes
representing the M variables. Use covariance matrix in other cases.

    """

    mean = np.mean(data, axis=0)

    data_adjust = data - mean

    #: the data is transposed due to np.cov/corrcoef syntax
    if correlation:

        matrix = np.corrcoef(data_adjust.T)

    else:
        matrix = np.cov(data_adjust.T)

    eigenvalues, eigenvectors = np.linalg.eig(matrix)

    if sort:
        #: sort eigenvalues and eigenvectors
        sort = eigenvalues.argsort()[::-1]
        eigenvalues = eigenvalues[sort]
        eigenvectors = eigenvectors[:, sort]

    return eigenvalues, eigenvectors


def best_fitting_plane(points, equation=False):
    """ Computes the best fitting plane of the given points

Parameters
----------        
points: array
    The x,y,z coordinates corresponding to the points from which we want
    to define the best fitting plane. Expected format:
        array([
        [x1,y1,z1],
        ...,
        [xn,yn,zn]])
        
equation(Optional) : bool
        Set the oputput plane format:
            If True return the a,b,c,d coefficients of the plane.
            If False(Default) return 1 Point and 1 Normal vector.    
Returns
-------
a, b, c, d : float
    The coefficients solving the plane equation.

or

point, normal: array
    The plane defined by 1 Point and 1 Normal vector. With format:
    array([Px,Py,Pz]), array([Nx,Ny,Nz])
    
    """

    w, v = PCA(points)

    #: the normal of the plane is the last eigenvector
    normal = v[:, 2]

    #: get a point from the plane
    point = np.mean(points, axis=0)

    if equation:
        a, b, c = normal
        d = -(np.dot(normal, point))
        return a, b, c, d

    else:
        return point, normal


In [71]:
a, b, c, d = best_fitting_plane(pts1, True)

inliers = []
outliers = []

plane_lenght = max(0.1, math.sqrt(a * a + b * b + c * c))
distance_ratio_threshold = 0.2
for idx, point in enumerate(pts1):
    x, y, z = point
    distance = math.fabs(a * x + b * y + c * z + d) / plane_lenght
    if distance <= distance_ratio_threshold:
        inliers.append(idx)
    else:
        outliers.append(idx)

inliers = pts1[inliers]
outliers_intens = intens[outliers]
outliers = pts1[outliers]

print(f"filtered {(inliers.shape[0]/number_of_ground_points) * 100 :.2f}% of ground")
mlab.points3d(outliers[:, 0], outliers[:, 1], outliers[:, 2], outliers_intens, mode='point')
mlab.show()


filtered 35.96% of ground


In [77]:
inliers_res, outliers_res = ransac(pts1, 20, 0.2, len(pts1)/5)
inliers = pts1[inliers_res]
outliers = pts1[outliers_res]
outliers_intens = intens[outliers_res]

In [82]:
print(f"filtered {(inliers.shape[0]/number_of_ground_points) * 100 :.2f}% of ground")
mlab.points3d(outliers[:, 0], outliers[:, 1], outliers[:, 2], outliers_intens, mode='point')
mlab.points3d(inliers[:, 0], inliers[:, 1], inliers[:, 2], color=(1,1,0), mode='point')

mlab.show()

filtered 78.88% of ground


In [88]:
inliers.shape

(76186, 3)

In [81]:
outliers.shape

(46022, 3)

In [90]:
inliers == ground_pts

/Users/vojtechbartek/opt/anaconda3/envs/bakalarka/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


False

In [ ]:
np.any(np.all(voxels4 == dynamic_voxel, axis=1))

In [95]:
def calculate_ground_removal_accuracy(ground_points, predicted_ground_points):
    correct = 0
    for point in predicted_ground_points:
        if np.any(np.all(ground_points == point, axis=1)):
            correct += 1
    
    num_of_predicted = predicted_ground_points.shape[0]
    tp = correct/ground_points.shape[0]
    fp = (num_of_predicted - correct) / num_of_predicted
    return tp,fp 

In [96]:
calculate_ground_removal_accuracy(inliers, ground_pts)

KeyboardInterrupt: 

## TODO oznacit zemi jako 1 a nezemi jako 0 a mit stejne velike pole jako original PCA


